In [1]:
import numpy as np
import tensorflow as tf
import os
#import imageio
import scipy.ndimage
import scipy.misc
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# Dataset


In [3]:
def imread(path, index, scale=1, gray=True, is_train=False):
    if gray:
        image = np.zeros([len(index), image_y, image_x, 1])
        cnt=0
        for i in (index):
            if is_train:
                img = scipy.misc.imread(path+str(i).zfill(4)+'x'+str(scale)+'.png', flatten=True, mode='YCbCr').astype(np.float)/255.
            else:
                img = scipy.misc.imread(path+str(i).zfill(4)+'.png', flatten=True, mode='YCbCr').astype(np.float)/255.
            if img.shape[1]<img.shape[0]:
                img=img.T
            if is_train and img.shape[0]>=540:
                image[cnt,:,:,0]=preprocess(img[:int(1080/scale),:int(1920/scale)], scale)
                cnt+=1
            elif is_train==False and img.shape[0]>=1080:
                image[cnt,:,:,0]=img[:int(1080/scale),:int(1920/scale)]
                cnt+=1
            
        return image[:cnt,:,:,:]
    else:
        image = np.zeros([len(index), image_y, image_x, 3])
        cnt=0
        for i in (index):
            if is_train:
                img = scipy.misc.imread(path+str(i).zfill(4)+'x'+str(scale)+'.png', mode='YCbCr').astype(np.float)/255.
            else:
                img = scipy.misc.imread(path+str(i).zfill(4)+'.png', mode='YCbCr').astype(np.float)/255.
            if img.shape[1]<img.shape[0]:
                img1=img[:,:,0].T
                img2=img[:,:,1].T
                img3=img[:,:,2].T
                img = np.stack([img1,img2,img3], axis=2)
            if is_train:
                image[cnt,:,:,:]=preprocess(img[:int(1080/scale),:int(1920/scale),:],scale)
            else:
                image[cnt,:,:,:]=img[:int(1080/scale),:int(1920/scale),:]
            cnt+=1
        return image

In [4]:
def preprocess(image, scale=2, gray=True):
    if gray:
        return scipy.ndimage.interpolation.zoom(image, (scale/1.), prefilter=False)
    else:
        image1 = scipy.ndimage.interpolation.zoom(image[:,:,0], (scale/1.), prefilter=False)
        image2 = scipy.ndimage.interpolation.zoom(image[:,:,1], (scale/1.), prefilter=False)
        image3 = scipy.ndimage.interpolation.zoom(image[:,:,2], (scale/1.), prefilter=False)
        imageA = np.stack([image1, image2, image3], axis=2)
        return imageA
    

In [5]:
def imsave(image, path):
    return scipy.misc.imsave(path, image)

# Networks

In [6]:
tf.reset_default_graph()
#device = "/gpu:0"

In [7]:
gray=True
epoch_size = 1000
batch_size = 2
total_iter = 800/batch_size
image_x = 1920
image_y = 1080

if gray==True:
    c_dim=1
else:
    c_dim=3
    
learning_rate = 1e-3

        

X = tf.placeholder(tf.float32, [None, image_y, image_x, c_dim], name='input')
Y = tf.placeholder(tf.float32, [None, image_y, image_x, c_dim], name='output')

weights = {
    'w1' : tf.get_variable('w1',shape=[3,3,1,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w2' : tf.get_variable('w2',shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w3' : tf.get_variable('w3',shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w4' : tf.get_variable('w4',shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w5' : tf.get_variable('w5',shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w6' : tf.get_variable('w6',shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w7' : tf.get_variable('w7',shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w8' : tf.get_variable('w8',shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'w9' : tf.get_variable('w9',shape=[3,3,64,1], initializer=tf.contrib.layers.xavier_initializer_conv2d())            
}
biases = {
    'b1' : tf.get_variable('b1',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b2' : tf.get_variable('b2',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b3' : tf.get_variable('b3',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b4' : tf.get_variable('b4',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b5' : tf.get_variable('b5',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b6' : tf.get_variable('b6',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b7' : tf.get_variable('b7',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b8' : tf.get_variable('b8',shape=[64], initializer=tf.contrib.layers.xavier_initializer_conv2d()),
    'b9' : tf.get_variable('b9',shape=[1], initializer=tf.contrib.layers.xavier_initializer_conv2d())
}

In [8]:
conv1 = tf.nn.relu(tf.nn.conv2d(X, weights['w1'], strides=[1,1,1,1], padding='SAME')+biases['b1'])
conv2 = tf.nn.relu(tf.nn.conv2d(conv1, weights['w2'], strides=[1,1,1,1], padding='SAME')+biases['b2'])
conv3 = tf.nn.relu(tf.nn.conv2d(conv2, weights['w3'], strides=[1,1,1,1], padding='SAME')+biases['b3'])
conv4 = tf.nn.relu(tf.nn.conv2d(conv3, weights['w4'], strides=[1,1,1,1], padding='SAME')+biases['b4'])
conv5 = tf.nn.relu(tf.nn.conv2d(conv4, weights['w5'], strides=[1,1,1,1], padding='SAME')+biases['b5'])
conv6 = tf.nn.relu(tf.nn.conv2d(conv5, weights['w6'], strides=[1,1,1,1], padding='SAME')+biases['b6'])
conv7 = tf.nn.relu(tf.nn.conv2d(conv6, weights['w7'], strides=[1,1,1,1], padding='SAME')+biases['b7'])
conv8 = tf.nn.relu(tf.nn.conv2d(conv7, weights['w8'], strides=[1,1,1,1], padding='SAME')+biases['b8'])
conv9 = tf.nn.conv2d(conv8, weights['w9'], strides=[1,1,1,1], padding='SAME')+biases['b9']
out_residual = tf.add(X,conv9)

loss = tf.reduce_mean(tf.square(Y-out_residual))
optm = tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [9]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=sess_config)
sess.run(tf.global_variables_initializer())

In [10]:
train_path='images/train_bicubic_x2/'
label_path='images/train_HR/'
result_path='results/SRCNN_v1/'

In [ ]:
for epoch in range(epoch_size):
    avr_psnr = 0
    for i in range(total_iter):

        index = np.random.choice(800, batch_size, replace=False)+1
        train_image = imread(path=train_path,index=index,is_train=True,scale=2)
        label_image = imread(path=label_path, index=index)

        sess.run(optm, feed_dict={X:train_image, Y:label_image})
        tr_loss = sess.run(loss, feed_dict={X:train_image, Y:label_image})
        psnr = 20*np.log10(1./np.sqrt(tr_loss))
        avr_psnr += psnr

    print ('epoch: %3d, Avr_PSNR: %4f' %(epoch, avr_psnr/total_iter))
    imgOUT = sess.run(out_residual, feed_dict={X:train_image})
    for j in range(imgOUT.shape[0]):
        imsave(imgOUT[j,:,:,0], result_path+'srcnn'+str(j).zfill(4)+'.png')
        imsave(train_image[j,:,:,0], result_path+'interpol_'+str(j).zfill(4)+'.png')



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


plt.imshow(train_image[3,:,:,0])